In [1]:
from astropy.io import fits
from astropy.time import Time
import numpy as np
from pyvo.dal import tap

In [2]:
# For scans - 46575 46576 46577 46578 46579 46580 46585 46586 46587 46588 46591
prov_ids = [
            ["APEXBOL.2022-08-24T12:46:40.000", "APEX-46575-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T12:54:33.000", "APEX-46576-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:00:39.000", "APEX-46577-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:13:50.000", "APEX-46578-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:27:16.000", "APEX-46579-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:40:27.000", "APEX-46580-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T13:58:26.000", "APEX-46585-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:11:39.000", "APEX-46586-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:24:53.000", "APEX-46587-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:38:05.000", "APEX-46588-2022-08-24-E-0110.C-4194A-2022"],
            # ["APEXBOL.2022-08-24T14:50:21.000", "APEX-46590-2022-08-24-E-0110.C-4194A-2022"],
            ["APEXBOL.2022-08-24T14:58:53.000", "APEX-46591-2022-08-24-E-0110.C-4194A-2022"]
            ]

prov_ids = np.array(prov_ids)

In [3]:
hdul = fits.open('./batch_30790/Orion_CONCERTO_ESO_2all_tot.fits')
hdul.info()

Filename: ./batch_30790/Orion_CONCERTO_ESO_2all_tot.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0                1 PrimaryHDU      49   ()      
  1  SP_CUBE       1 ImageHDU        84   (360, 360, 401)   float32   
  2  SP_NOISE      1 ImageHDU        89   (360, 360, 401)   float32   


In [4]:
header_0 = hdul[0].header
header_1 = hdul[1].header
header_2 = hdul[2].header

In [5]:
# Make TAP query to get observation details
def tap_query(query, ESO_TAP_OBS="https://archive.eso.org/tap_obs"):
    """Function to perform a TAP query to the ESO archive."""
    tapobs = tap.TAPService(ESO_TAP_OBS)
    result = tapobs.search(query=query, maxrec=1000).to_qtable()
    return result

prog_id = "0110.C-4194(A)"
object_name = "Orion"

query = f"""SELECT dp_id, exposure, prog_id, object, dp_tech, instrument, ra, dec, exp_start, origfile
            FROM dbo.raw
            WHERE dp_id like 'APEXBOL.%%'
                AND object = '{object_name}'
                AND prog_id = '{prog_id}'
                AND dp_cat = 'SCIENCE'""" 

result = tap_query(query)

# Mask out scan not used (i.e. only 11 out of 12 were used)
# 46590 not used 
mask = []
for origfile in result["origfile"]:
    if "46590" in origfile:
        mask.append(False)
    else:
        mask.append(True)
result = result[mask]

# Get the first and last exposure start times
exp_start = result["exp_start"]
exposure = result["exposure"]
dp_id = result["dp_id"]

# Get total exposure time 
texptime = float(np.sum(exposure).to("s").value)

mjdobs = exp_start[0]
mjdend = exp_start[-1]
dateobs = mjdobs.replace("Z", "")

mjdobs = Time(mjdobs, scale="utc")
mjdend = Time(mjdend, scale="utc")

mjdobs = float(round(mjdobs.mjd, 5))
mjdend = round(float(mjdend.mjd) + float(exposure[-1].to("d").value), 5)

result

dp_id,exposure,prog_id,object,dp_tech,instrument,ra,dec,exp_start,origfile
,s,,,,,deg,deg,,
object,float32,object,object,object,object,float64,float64,object,object
APEXBOL.2022-08-24T12:46:40.000,330.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T12:46:39.100Z,APEX-46575-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T12:54:33.000,360.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T12:54:32.100Z,APEX-46576-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:00:39.000,630.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:00:39Z,APEX-46577-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:13:50.000,720.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:13:50Z,APEX-46578-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:27:16.000,630.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:27:16Z,APEX-46579-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:40:27.000,720.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:40:26.100Z,APEX-46580-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T13:58:26.000,630.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T13:58:26Z,APEX-46585-2022-08-24-E-0110.C-4194A-2022
APEXBOL.2022-08-24T14:11:39.000,720.0,0110.C-4194(A),ORION,CONTINUUM,APEXBOL,83.82208305,-5.39111111,2022-08-24T14:11:38.100Z,APEX-46586-2022-08-24-E-0110.C-4194A-2022


In [6]:
del header_0["PSF_FWHM"]
del header_0["PHOTZP"]
del header_0["PHOTSYS"]
del header_0["ABMAGLIM"]
del header_0["ABMAGSAT"]
del header_0["COMMENT"]

header_0.set("PRODCATG", "SCIENCE.CUBE", "Data product category", after="EXTEND")
header_0.set("BMAJ", header_0["SKY_RES"]/3600., "Major axis beam size (deg)")
header_0.set("BMIN", header_0["SKY_RES"]/3600., "Minor axis beam size (deg)")
header_0.set("BPA", 0.0, "Position angle of the beam (deg)")
header_0.set("INSTRUME", "APEXBOL", "Instrument type")
header_0.set("FEBE1", "CONCERTO-CONCERTOBE", "Frontend-Backend")
header_0.set("TIMESYS", "TAI", "Time system for MJD")
header_0.set("BNOISE", 0.0, "Median rms (Jy)")
header_0.set("FLUXERR", 0.15, "Flux error (15%)")
header_0.set("PROG_ID", prog_id, "Program ID")
header_0.set("MJD-OBS", mjdobs, "MJD of observation") # From raw file header... 
header_0.set("MJD-END", mjdend, "MJD of end of observation")
header_0.set("TEXPTIME", texptime, "Total exposure time (s)")
header_0.set("EXPTIME", texptime, "Total exposure time (s)")

header_0.set("DATE-OBS", dateobs, "Date of observation in ISO format")
date = Time.now().iso.replace(" ", "T")
header_0.set("DATE", date, "Date of file creation in ISO format")

header_0.set("OBSTECH", "SPECTRUM", "Technique of observation")

for i, prov_id in enumerate(prov_ids[:,0]):
    header_0.set(f"PROV{i+1}", prov_id, f"Provenance ID {i+1}")

header_0.set("ASSON1", "Orion_CONCERTO_ESO_2all_whitelight.fits", "Whitelight image")

header_0

SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                   32 /                                                
NAXIS   =                    0 /                                                
EXTEND  =                    T /Extensions may be present                       
PRODCATG= 'SCIENCE.CUBE'       / Data product category                          
ORIGIN  = 'APEX    '           /                                                
TELESCOP= 'APEX-12m'           /                                                
INSTRUME= 'APEXBOL '           / Instrument type                                
OBJECT  = 'Orion   '           /                                                
EQUINOX =              2000.00 /                                                
EXPTIME =               6810.0 / Total exposure time (s)                        
TEXPTIME=               6810.0 / Total exposure time (s)                        
MJD-OBS =           59815.53

In [7]:
del header_1["FLUXERR"]
del header_1["PHOTZP"]
del header_1["PHOTSYS"]
del header_1["ABMAGLIM"]
del header_1["ABMAGSAT"]
del header_1["SKY_RES"]
del header_1["PSF_FWHM"]
del header_1["SPEC_RES"]
del header_1["WAVELMIN"]
del header_1["WAVELMAX"]
del header_1["PROCSOFT"]
del header_1["FLUXCAL"]
del header_1["TEXPTIME"]
del header_1["EXPTIME"]
del header_1["EQUINOX"]
del header_1["INSTRUME"]
del header_1["TELSCOP"]
del header_1["ORIGIN"]
del header_1["MJD-OBS"]
del header_1["MJD-END"]
del header_1["PROG_ID"]
del header_1["REFERENC"]
del header_1["DATE"]
del header_1["DATE-OBS"]
del header_1["NCOMBINE"]
del header_1["PROV1"]
del header_1["OBSTECH"]
del header_1["MAPMODE"]
del header_1["BNOISE"]
del header_1["FEBE1"]
del header_1["Comment"]
del header_1["History"]
del header_1["ASSON1"]
del header_1["QUALDATA"] # No quality flags in data cube... 

header_1.set("CTYPE3", "FREQ")
header_1.set("EXTNAME", "DATA_EXT")
header_1.set("ERRDATA", "STAT_EXT")

header_1

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                  360 /                                                
NAXIS2  =                  360 /                                                
NAXIS3  =                  401 /                                                
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
WCSAXES =                    3 / 2 Coordinates, 1 frequency scale               
CRPIX1  =        180.000000000 / Pixel coordinate of reference point            
CRPIX2  =        180.000000000 / Pixel coordinate of reference point            
CRPIX3  =        1.00000000000 /                                                
CUNIT1  = 'deg'             

In [8]:
del header_2["FLUXERR"]
del header_2["PHOTZP"]
del header_2["PHOTSYS"]
del header_2["ABMAGLIM"]
del header_2["ABMAGSAT"]
del header_2["SKY_RES"]
del header_2["PSF_FWHM"]
del header_2["SPEC_RES"]
del header_2["WAVELMIN"]
del header_2["WAVELMAX"]
del header_2["PROCSOFT"]
del header_2["FLUXCAL"]
del header_2["TEXPTIME"]
del header_2["EXPTIME"]
del header_2["EQUINOX"]
del header_2["INSTRUME"]
del header_2["TELSCOP"]
del header_2["ORIGIN"]
del header_2["MJD-OBS"]
del header_2["MJD-END"]
del header_2["PROG_ID"]
del header_2["REFERENC"]
del header_2["DATE"]
del header_2["DATE-OBS"]
del header_2["NCOMBINE"]
del header_2["PROV1"]
del header_2["OBSTECH"]
del header_2["MAPMODE"]
del header_2["BNOISE"]
del header_2["FEBE1"]
del header_2["Comment"]
del header_2["History"]
del header_2["ASSON1"]
del header_2["QUALDATA"] # No quality flags in data cube... 
del header_2["ERRDATA"] 

header_2.set("CTYPE3", "FREQ")
header_2.set("EXTNAME", "STAT_EXT")
header_2.set("HDUCLAS3", "RMSE")
header_2.set("SCIDATA", "DATA_EXT")

header_2

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                  360 /                                                
NAXIS2  =                  360 /                                                
NAXIS3  =                  401 /                                                
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
WCSAXES =                    3 / 2 Coordinates, 1 frequency scale               
CRPIX1  =        180.000000000 / Pixel coordinate of reference point            
CRPIX2  =        180.000000000 / Pixel coordinate of reference point            
CRPIX3  =        1.00000000000 /                                                
CUNIT1  = 'deg'             

In [9]:
# Convert frequency units in HDU headers from Hz to GHz
converted_hdus = []
for i, hdu in enumerate(hdul):
    hdr = hdu.header
    cunit3 = hdr.get('CUNIT3')
    if cunit3 and 'HZ' in str(cunit3).strip().upper():
        # Convert common frequency-related keywords if present
        for key in ('CRVAL3', 'CDELT3', 'CD3_3'):
            if key in hdr:
                try:
                    val = hdr[key]
                    if isinstance(val, (int, float)):
                        hdr.set(key, val / 1e9, f"Converted {key} from Hz to GHz")
                except Exception:
                    # skip non-numeric or unexpected values
                    pass
        hdr.set('CUNIT3', 'GHz', 'Frequency unit converted from Hz to GHz')
        converted_hdus.append(i)

print('Converted CUNIT3 -> GHz in HDU indices:', converted_hdus)

Converted CUNIT3 -> GHz in HDU indices: [1, 2]


In [10]:
def trim_cube_to_ghz(hdul, fmin_ghz, fmax_ghz, verbose=True):
    """
    Trim spectral axis of a FITS HDUList to the frequency range [fmin_ghz, fmax_ghz] (GHz).
    - Finds the first HDU that contains a spectral axis (NAXIS3) and uses it as the reference.
    - Slices data arrays (along axis 0) for any HDU whose first axis length matches the reference spectral length.
    - Updates per-HDU header keywords: NAXIS3, CRPIX3, CRVAL3, CDELT3/CD3_3 and sets CUNIT3 -> 'GHz'.
    - Updates primary HDU (hdul[0]) WAVELMIN/WAVELMAX in nm based on the kept frequency range.

    Returns: (hdul, info)
    """
    import numpy as np

    # Find reference HDU (first HDU with a non-zero NAXIS3)
    ref_idx = None
    for i, hdu in enumerate(hdul):
        hdr = hdu.header
        if 'NAXIS3' in hdr and int(hdr['NAXIS3']) > 0:
            ref_idx = i
            break
    if ref_idx is None:
        raise ValueError('No HDU with NAXIS3 found to use as reference')

    ref_hdr = hdul[ref_idx].header
    N = int(ref_hdr['NAXIS3'])
    ref_crpix = float(ref_hdr.get('CRPIX3', 1.0))
    ref_crval = float(ref_hdr.get('CRVAL3', 0.0))

    # Determine delta keyword for the reference
    if 'CDELT3' in ref_hdr:
        delta_key = 'CDELT3'
    elif 'CD3_3' in ref_hdr:
        delta_key = 'CD3_3'
    else:
        raise ValueError('Neither CDELT3 nor CD3_3 found in reference header')
    ref_delta = float(ref_hdr[delta_key])

    # Unit handling: convert header values to GHz for the pixel->freq calculation
    unit = str(ref_hdr.get('CUNIT3', '')).strip().upper()
    if 'HZ' in unit and 'GHZ' not in unit:
        to_ghz = 1e-9
    elif 'GHZ' in unit:
        to_ghz = 1.0
    elif unit == '' or unit is None:
        # assume Hz if missing (conservative)
        to_ghz = 1e-9
        if verbose:
            print('Warning: CUNIT3 missing in reference header; assuming Hz')
    else:
        to_ghz = 1.0
        if verbose:
            print(f'Warning: unrecognized CUNIT3="{unit}"; treating header values as GHz')

    # Build frequency array in GHz for pixel indices 1..N (FITS 1-based pixels)
    pix = np.arange(1, N+1)
    freqs_ghz = (ref_crval + (pix - ref_crpix) * ref_delta) * to_ghz

    # Ensure fmin <= fmax
    fmin, fmax = float(min(fmin_ghz, fmax_ghz)), float(max(fmin_ghz, fmax_ghz))

    mask = (freqs_ghz >= fmin) & (freqs_ghz <= fmax)
    if not mask.any():
        raise ValueError(f'No spectral pixels found in range {fmin}--{fmax} GHz')

    # Find contiguous slice covering the requested range (first..last True)
    start_idx = int(np.argmax(mask))        # zero-based index of first True
    end_idx = int(len(mask) - 1 - np.argmax(mask[::-1]))  # zero-based last True
    new_len = end_idx - start_idx + 1

    if verbose:
        print(f'Reference HDU index: {ref_idx}, original channels: {N}')
        print(f'Trimming spectral axis: keeping pixels {start_idx+1}..{end_idx+1} (1-based), {new_len} channels')

    # For each HDU, slice data if present and update header keywords where appropriate
    for hdu in hdul:
        hdr = hdu.header
        # Only proceed if header has NAXIS3
        if 'NAXIS3' not in hdr or int(hdr['NAXIS3']) == 0:
            # skip non-spectral HDUs
            continue

        old_N = int(hdr['NAXIS3'])

        # Slice data if data exists and first axis matches reference N
        if getattr(hdu, 'data', None) is not None:
            try:
                data = hdu.data
                if data.ndim >= 1 and data.shape[0] == N:
                    # slice along the spectral axis (axis 0 in FITS order)
                    hdu.data = data[start_idx:end_idx+1, ...]
                else:
                    if verbose:
                        print(f'Note: HDU "{hdr.get("EXTNAME", "")}" data shape {getattr(hdu, "data", None).shape if getattr(hdu, "data", None) is not None else None} does not have spectral axis length {N}; skipping data slice')
            except Exception as e:
                if verbose:
                    print('Warning slicing HDU data:', e)

        # Update header numeric keywords: convert to GHz if needed, and adjust CRPIX3/NAXIS3
        # Per-HDU old_crpix and old_crval
        try:
            old_crpix = float(hdr.get('CRPIX3', ref_crpix))
        except Exception:
            old_crpix = ref_crpix
        try:
            # Convert existing CRVAL3 to GHz (it is the value at CRPIX3)
            if 'CRVAL3' in hdr:
                hdr['CRVAL3'] = float(hdr['CRVAL3']) * to_ghz
        except Exception:
            pass

        # Set new CRPIX3 relative to the sliced cube
        try:
            hdr['CRPIX3'] = old_crpix - start_idx
        except Exception:
            pass

        # Convert and/or set deltas
        if 'CDELT3' in hdr:
            try:
                hdr['CDELT3'] = float(hdr['CDELT3']) * to_ghz
            except Exception:
                pass
        if 'CD3_3' in hdr:
            try:
                hdr['CD3_3'] = float(hdr['CD3_3']) * to_ghz
            except Exception:
                pass

        # Update unit and NAXIS3
        hdr['CUNIT3'] = 'GHz'
        hdr['NAXIS3'] = new_len

    # Update primary WAVELMIN/WAVELMAX (in nm) based on kept frequency range
    try:
        primary_hdr = hdul[0].header
        fmin_kept = float(freqs_ghz[start_idx])
        fmax_kept = float(freqs_ghz[end_idx])
        # wavelengths in nm: lambda_nm = c (m/s) / freq_GHz
        if fmin_kept <= 0 or fmax_kept <= 0:
            if verbose:
                print('Warning: non-positive frequency encountered; skipping WAVELMIN/WAVELMAX update')
        else:
            lambda_min_nm = 299792458.0 / fmax_kept
            lambda_max_nm = 299792458.0 / fmin_kept
            primary_hdr.set('WAVELMIN', lambda_min_nm, '[nm] Minimum wavelength')
            primary_hdr.set('WAVELMAX', lambda_max_nm, '[nm] Maximum wavelength')
            if verbose:
                print(f'Updated primary WAVELMIN={lambda_min_nm:.6g} nm, WAVELMAX={lambda_max_nm:.6g} nm')
    except Exception as e:
        if verbose:
            print('Warning updating primary WAVELMIN/WAVELMAX:', e)

    info = dict(ref_hdu=ref_idx, start_pixel=start_idx+1, end_pixel=end_idx+1, n_channels=new_len, fmin_ghz=float(freqs_ghz[start_idx]), fmax_ghz=float(freqs_ghz[end_idx]))
    if verbose:
        print('Trim result:', info)
    return hdul, info

hdul, info = trim_cube_to_ghz(hdul, 90.0, 400.0)

Reference HDU index: 1, original channels: 401
Trimming spectral axis: keeping pixels 32..134 (1-based), 103 channels
Updated primary WAVELMIN=751880 nm, WAVELMAX=3.22581e+06 nm
Trim result: {'ref_hdu': 1, 'start_pixel': 32, 'end_pixel': 134, 'n_channels': 103, 'fmin_ghz': 92.93566197999999, 'fmax_ghz': 398.72396913999995}


In [11]:
hdul.writeto('./batch_30931/Orion_CONCERTO_ESO_2all_tot.fits', checksum=True, overwrite=True)